## Monday Meeting 


I have adapted the MLP dataprovider to the accelerometer data set to be used. I found the NN coded by Pawel extremely well documented and useful and thus I have picked it over Lasagne, tensorflow or the horrendous scikit learn (on which I wasted a whole day pointlessly).


In [11]:
import numpy
import logging
from mlp.dataset import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = ACLDataProvider(dset='train', batch_size=100, max_num_batches=-10, randomize=True)
valid_dp = ACLDataProvider(dset='valid', batch_size=1824, max_num_batches=1, randomize=False)

INFO:root:Initialising data providers...


(7296, 500) (7296,)
(1824, 500) (1824,)


In [17]:
#Baseline experiment
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

#some hyper-parameters
nhid = 100
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = list()

test_dp = deepcopy(valid_dp)
train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(Sigmoid(idim=500, odim=500, irange=1.6, rng=rng))
model.add_layer(Softmax(idim=500, odim=19, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')

tst_cost, tst_accuracy = optimiser.validate(model,test_dp )
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))



INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 3.373. Accuracy is 3.44%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 3.400. Accuracy is 3.18%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 5.476. Accuracy is 41.90%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 2.584. Accuracy is 47.09%
INFO:mlp.optimisers:Epoch 1: Took 1 seconds. Training speed 12002 pps. Validation speed 30417 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 1.744. Accuracy is 65.03%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 0.723. Accuracy is 78.29%
INFO:mlp.optimisers:Epoch 2: Took 1 seconds. Training speed 10436 pps. Validation speed 30417 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 1.147. Accuracy is 73.22%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 0.919. Accuracy is 75.82%
INFO:mlp.optimisers:Epoch 3: Took 1 seconds. Training speed 12002 pps. Validation speed 30417 pps.
INFO:mlp.optimi